## Load data

In [2]:
import json

with open('./data/darkstar.json', 'r') as fp:
    d = json.load(fp)

In [3]:
d.keys()

dict_keys(['nodes', 'edges', 'parameters'])

## Util

In [60]:
def save(path, s):
    with open(path, 'w') as fp:
        fp.write(s)

## Graph

### Nodes

In [61]:
_s = ','.join([f"'{k}'" for k in d['nodes'].keys()])
_s = f'c({_s})'
_s = f'g <- empty.graph(nodes={_s})'

save('nodes.R', _s)

### Edges

In [62]:
_s = [f'g <- set.arc(g, from="{d["parent"]}", to="{d["child"]}")' for d in d['edges']]
_s = '\n'.join(_s)

save('edges.R', _s)

## Parameters

### Values

In [63]:
def get_values(district):
    values = [f'"{v}"' for v in d['nodes'][district]['values'].keys()]
    values = ', '.join(values)
    values = f'c({values})'
    values = f'{district}.lv <- {values}'
    return values

_s = [get_values(k) for k in d['nodes'].keys()]
_s = '\n'.join(_s)

save('values.R', _s)

### Probabilities

In [64]:
def get_dim(district):
    cols = len(d['nodes'][district]['values'])
    rows = len(d['parameters'][district])
    
    if rows == cols:
        s = f'{cols}'
    else:
        rows = int(rows / 2)
        s = f'c({rows}, {cols})'
    
    return s

def get_dimnames(district):
    r = d['parameters'][district][0]
    r = [k for k in r if k != '__p__']
    r = [f'{k}={k}.lv' for k in r]
    r = ', '.join(r)
    r = f'list({r})'
    return r

def get_p(district):
    r = d['parameters'][district]
    r = [_r['__p__'] for _r in r]
    r = [f'{_r}' for _r in r]
    r = ', '.join(r)
    r = f'c({r})'
    
    return r

def get_prob(district):
    dim = get_dim(district)
    dimnames = get_dimnames(district)
    p = get_p(district)
    
    return f'{district}.prob <- array({p}, dim={dim}, dimnames={dimnames})'

# get_dim('AZ_02'), get_dimnames('AZ_02'), get_p('AZ_02')
# get_dim('CA_12'), get_dimnames('CA_12'), get_p('CA_12')

_s = [get_prob(k) for k in d['nodes'].keys()]
_s = '\n'.join(_s)

save('probabilities.R', _s)